# WFiApp

Aplikacja będzie wykorzystywać `LazyColumn`, gdzie każdym elementem na liście będzie `Card`. Dodamy również podstawową obsługę gestów oraz wykorzystamy `Compose Navigation` aby otworzyć nowy ekran w którym zaprezentowany będzie bardziej szczegółowy opis wybranego elementu listy. Będzie to wersja WFiApp z poprzedniego modułu, tym razem wykonana w `Compose`.

<table><tr><td><img src="https://media2.giphy.com/media/gooBI30gtU5fMkmCZu/giphy.gif?cid=790b7611df027999cb2a17ec28a3095ad5693e6e76c37990&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media2.giphy.com/media/h5Xh7V5IA5E3BV4ytL/giphy.gif?cid=790b7611412fedb227b91c2cb44067aa88f517a18214afde&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/uwDAUUAjLZnyKqLg3J/giphy.gif?cid=790b76110bdb83e20fb638b9fb5150422cf100e384687bcf&rid=giphy.gif&ct=g" width="150" /></td></tr></table>

Będziemy wykorzystywać `Compose Navigation`, więc dodajmy odpowiednią zależność.

In [ ]:
implementation 'androidx.navigation:navigation-compose:2.5.3'

## Model

Dodajmy model reprezentujący instytut

In [ ]:
data class Institute(
    val title: String,
    val info: String,
    val imageResource: Int // identyfikatory obrazów są przechowywane jako int
)

Następnie dodajmy obiekt `DataProvider` przechowujący listę wszystkich instytutów

In [ ]:
object DataProvider {

    val institutes: ArrayList<Institute> = ArrayList()

    fun getInstituteData(activity: Activity){
        val instituteList = activity.resources.getStringArray(R.array.institute_titles)
        val instituteInfo = activity.resources.getStringArray(R.array.institute_info)
        val instituteImageResources = activity.resources.obtainTypedArray(R.array.institute_images)

        for (i in instituteList.indices) institutes.add(
            Institute(
                instituteList[i],
                instituteInfo[i],
                instituteImageResources.getResourceId(i, 0)
            )
        )

        instituteImageResources.recycle()
    }
}

## Lista - `Card` + `LazyColumn`

<img src="https://i.ibb.co/PgKQ6nS/image.png" width="350" />

Chcemy wyświewtlić listę wszystkich instytutów - każde pola będzie zawierać grafikę, tytuł znajdujący się na grafice (wraz z gradientem, aby poprzwić widoczność tekstu), oraz pole tekstowe zawierające podstawowe informacje. Rozpocznijmy od utworzenia naszej karty, dodajmy funkcję `Composable`, która renderuje kartę z obrazem.

In [ ]:
@Composable
fun ImageCard(
    painter: Painter,
    contentDescription: String,
    title: String,
    info: String,
    modifier: Modifier = Modifier
){

ImageCard przyjmuje kilka parametrów:
- `painter` - obiekt typu `Painter`, który reprezentuje obraz do wyświetlenia.
- `contentDescription` - opis zawartości obrazu, który jest używany w celach dostępności.
- `title` - tytuł karty, który będzie wyświetlany.
- `info` - dodatkowe informacje na temat karty, które będą wyświetlane.
- `modifier` - modyfikator, który kontroluje wygląd i rozmiar karty. Jest opcjonalny i domyślnie ustawiony na `Modifier`.

Wewnątrz funkcji zdefiniujmy `Card`

In [ ]:
Card(
    modifier = modifier
        .fillMaxWidth()
        .padding(15.dp)
        .clickable { },
    shape = RoundedCornerShape(15.dp),
    elevation = CardDefaults.cardElevation(defaultElevation = 10.dp)
) {}

`Card` jest komponentem, który używamy do tworzenia pojemników na inne komponenty, które mogą być stylizowane jako karty. Przyjmuje kilka parametrów, które definiują jego wygląd i zachowanie:
- `modifier` - modyfikator który kontroluje wygląd i rozmiar karty. W tym przypadku, modyfikator składa się z kilku operacji, takich jak `fillMaxWidth()` (wypełnienie karty na szerokość dostępnej przestrzeni), `padding(15.dp)` (dodanie wewnętrznego marginesu 15 jednostek) i `clickable {}` (ustawienie obsługi kliknięcia na kartę).
- `shape` - kształt karty. W tym przypadku, używany jest `RoundedCornerShape(15.dp)`, co oznacza, że karta będzie miała zaokrąglone narożniki o promieniu 15 jednostek.
- `elevation` - podwyższenie karty. W tym przypadku, używany jest `CardDefaults.cardElevation(defaultElevation = 10.dp)`, co oznacza, że karta będzie miała domyślne podwyższenie o wartości 10 jednostek.

Ponieważ sama grafika, kolor pod tekstem oraz tekst znajdują się na stosie (jedne na drugich), wewnątrz `Card` definiujemy `Box` - pozwala dodawać elementy na stos (podobnie jak `Column` dodaje elementuy jeden pod drugim i `Row` jeden obok drugiego, `Box` dodaje elementy jeden na drugi).

In [ ]:
Box(modifier = Modifier
    .height(200.dp)
    .fillMaxWidth()
   ){}

Wysokość `Box` definiujemy na `200.dp` i wypełniamy całą dostępną szerokość. Wewnątrz chcemy umieścić trzy elementy, rozpocznijmy od dodania grafiki. Zdefiniujmy funkcję `Composable` `InstituteImage`

In [ ]:
@Composable
private fun InstituteImage(
    painter: Painter,
    contentDescription: String
) {
    Image(
        painter = painter,
        contentDescription = contentDescription,
        contentScale = ContentScale.Crop,
        modifier = Modifier.fillMaxWidth()
    )
}

`InstituteImage` przyjmuje dwa parametry:
- `painter` - obiekt typu `Painter`, który reprezentuje obraz do wyświetlenia.
- `contentDescription` - opis zawartości obrazu, który jest używany w celach dostępności.
Wewnątrz `InstituteImage` używamy komponentu `Image`, który renderuje obrazek. Przekazujemy `painter` jako parametr do `Image`, który reprezentuje obraz do wyświetlenia, a `contentDescription` jako opis dostępności dla obrazu. Dodatkowo, ustawiamy `contentScale` na `ContentScale.Crop`, co oznacza, że obrazek będzie przycięty, aby zmieścić się w ramach komponentu, jednocześnie zachowując proporcje. Modyfikator `Modifier.fillMaxWidth()` został zastosowany do komponentu `Image`, co oznacza, że obrazek wypełni całą dostępną szerokość.

Drugim elementem będzie gradient koloru, który będzie wyświetlany pod tekstem. Dodajmy funkcję `Composable` `Gradient`

In [ ]:
@Composable
private fun Gradient() {
    Box(
        modifier = Modifier
            .fillMaxSize()
            .background(
                Brush.verticalGradient(
                    colors = listOf(
                        Color.Transparent,
                        Color.Black
                    ),
                    startY = 450f
                )
            )
    )
}

Wewnątrz funkcji `Gradient` używamy komponentu `Box`, który działa jako kontener dla innych komponentów. W modyfikatorze `modifier` komponentu `Box`, zastosowano modyfikator `Modifier.fillMaxSize()`, który sprawia, że `Box` wypełnia całą dostępną przestrzeń. Użyto modyfikatora `.background()`, aby ustawić tło `Box` jako gradient. Wewnątrz modyfikatora `.background()`, użyto `Brush.verticalGradient()`, który tworzy pionowy gradient. W parametrze `colors` metody `Brush.verticalGradient()`, przekazano listę kolorów, w tym `Color.Transparent` i `Color.Black`. Gradient będzie płynnie przechodził od przezroczystego koloru na górze do czarnego koloru na dole. Dodatkowo, użyto parametru `startY = 450f`, który definiuje początek gradientu od 450 pikseli od góry.

Ostatnim elementem jest `Text` wyświetlający tytuł

In [ ]:
Text(
    text = title,
    modifier = Modifier
        .align(Alignment.BottomStart)
        .padding(12.dp),
    fontSize = 24.sp,
    style = TextStyle(color = Color.White)
)

Pod komponentem `Box`, w którym umieszczamy grafikę, gradient oraz tytuł, dodajmy jeszcze jeden komponent wyświetlający `info`

In [ ]:
Box(modifier = Modifier
    .fillMaxWidth()
    .height(45.dp)
) {
    Text(text = info)
}

Pełny kod funkcji `ImageCard`

In [ ]:
@Composable
fun ImageCard(
    painter: Painter,
    contentDescription: String,
    title: String,
    info: String,
    modifier: Modifier = Modifier
){
        Card(
            modifier = modifier
                .fillMaxWidth()
                .padding(15.dp)
                .clickable { },
            shape = RoundedCornerShape(15.dp),
            elevation = CardDefaults.cardElevation(defaultElevation = 10.dp)
        ) {
            Box(modifier = Modifier
                .height(200.dp)
                .fillMaxWidth()){
                InstituteImage(painter, contentDescription)
                Gradient()
                Text(
                    text = title,
                    modifier = Modifier
                        .align(Alignment.BottomStart)
                        .padding(12.dp),
                    fontSize = 24.sp,
                    style = TextStyle(color = Color.White)
                )
            }
            Box(modifier = Modifier
                .fillMaxWidth()
                .height(45.dp)
            ) {
                Text(text = info)
            }
        }
}

Dodajmy funkcję `InstituteList` wyświetlającą wszystkie elementy listy danych z `DataProvider` za pomocą `ImageCard`

In [ ]:
@Composable
fun InstituteList(){
    val data by remember {
        mutableStateOf(DataProvider.institutes.toList())
    }

    LazyColumn(
        modifier = Modifier.fillMaxSize()
    ){
        items(data.size){index ->
            Row(
                modifier = Modifier
                    .fillMaxSize()
                    .padding(4.dp)
                    .clickable { /*TODO*/ },
                horizontalArrangement = Arrangement.SpaceAround,
                verticalAlignment = Alignment.CenterVertically
            ) {
                ImageCard(
                    painter = painterResource(id = data[index].imageResource),
                    contentDescription = data[index].title,
                    title = data[index].title,
                    info = data[index].info
                )
            }
        }
    }
}

`InstituteList` jest funkcją komponującą, która renderuje listę instytutów. Używamy `mutableStateOf` w celu utworzenia zmiennej stanu `data`, która jest zmienną, przechowującą listę instytutów. Wywołanie `DataProvider.institutes.toList()` zwraca listę instytutów z dostawcy danych.

Wewnątrz `LazyColumn`, używamy bloku `items` do iteracji po każdym elemencie `data` i generowania wierszy w liście. Używamy komponentu `Row`, który renderuje wiersz z komponentami ułożonymi w poziomie. W modyfikatorze `Row`, używamy `Modifier.fillMaxSize()`, który sprawia, że Row wypełnia całą dostępną przestrzeń. Dodatkowo, używamy modyfikatora `.padding(4.dp)` do dodania marginesu 4 jednostek wokół wiersza. Używamy modyfikatora `.clickable { /*TODO*/ }`, aby dodać obsługę kliknięcia wiersza. Zrobimy to na późniejszym etapie. Wewnątrz `Row` używamy komponentu `ImageCard` (zdefiniowanego wcześniej), który renderuje kartę z obrazkiem dla każdego instytutu. Przekazujemy odpowiednie parametry do `ImageCard`, takie jak `painter` (obraz do wyświetlenia), `contentDescription` (opis obrazu), title (tytuł instytutu) i `info` (informacje o instytucie).

Fuinkcję `InteitutList` możemy wywołać w `MainActivity`

<img src="https://media4.giphy.com/media/Acr6sC2y1H1SY2pxEj/giphy.webp" width="200" />